# DATA CLEARNING FOR ZFC

In [1]:
%matplotlib inline

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

## Tool functions preparation

### Combine base and knowledge data

In [2]:
def combined_base_knowledge_data(base_data, knowledge_data):
    combined_base_knowledge_data = base_data.set_index('ID').join(knowledge_data.set_index('ID'))
    combined_base_knowledge_data = combined_base_knowledge_data.fillna(combined_base_knowledge_data.median())
    return combined_base_knowledge_data

### Function definition for year_fill_na

In [3]:
# fill null year 
def year_fill_na(oragin_data, years = [2015, 2016, 2017]):
    # pick null year data
    null_years = oragin_data.loc[oragin_data.year.isna()]
    
    # fill year 
    for year in years:
        IDs = oragin_data[["ID"]].loc[oragin_data.year == year]
        for null_year_id in null_years["ID"].unique():
            tmp = IDs.loc[IDs.ID == null_year_id]
            if tmp.empty:
                index = oragin_data.loc[(oragin_data.ID == null_year_id) 
                                        & (oragin_data.year.isna())].index.tolist()
                if len(index) != 0:
                    oragin_data.loc[index[0]:index[0], "year"] = year 
                
    new_data = oragin_data.fillna(oragin_data.median())
    return new_data;

### Function definition for combined_new_year_money_data

In [4]:
def combined_new_year_money_data(new_year_data, new_money_data):
    return pd.merge(new_year_data, new_money_data, on=['ID', 'year'])

### Split year_money_data

In [5]:
def split_data(combined_new_year_money_data):
    # split data with year
    combined_new_year_data_2015 = combined_new_year_money_data.loc[
        combined_new_year_money_data.year == 2015].set_index('ID').add_suffix("_2015").drop(columns=['year_2015'])
    combined_new_year_data_2016 = combined_new_year_money_data.loc[
        combined_new_year_money_data.year == 2016].set_index('ID').add_suffix("_2016").drop(columns=['year_2016'])
    combined_new_year_data_2017 = combined_new_year_money_data.loc[
        combined_new_year_money_data.year == 2017].set_index('ID').add_suffix("_2017").drop(columns=['year_2017'])
    
    # marge data with ID
    combined_new_splited_year_money_data = pd.merge(combined_new_year_data_2015, 
                                                    combined_new_year_data_2016, 
                                                    on=['ID'])
    combined_new_splited_year_money_data = pd.merge(combined_new_splited_year_money_data, 
                                                    combined_new_year_data_2017, 
                                                    on=['ID'])

    return combined_new_splited_year_money_data

## Training Data Set

### Load original training data

In [6]:
import pandas as pd

In [7]:
# load training data
base_train_data = pd.read_csv("data/train/base-train.csv")
year_train_data = pd.read_csv("data/train/year-train.csv")
knowledge_train_data = pd.read_csv("data/train/knowledge-train.csv")
money_train_data = pd.read_csv("data/train/money-train.csv")

### base and knowledge data

In [8]:
base_train_data.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,5986361,2014.0,7090.0,服务业,湖北,有限责任公司,自然人,0.93,0
1,5991749,2007.0,5940.0,零售业,湖南,合伙企业,企业法人,0.57,0
2,5998154,2002.0,9720.0,工业,福建,合伙企业,自然人,0.74,0
3,5984390,2000.0,4800.0,商业服务业,山东,股份有限公司,NaN,0.90,0
4,5980535,2004.0,4530.0,零售业,广东,农民专业合作社,自然人,0.95,0


In [9]:
base_train_data.describe()

,ID,注册时间,注册资本,控制人持股比例,flag
count,2.851900e+04,28230.000000,28220.000000,28223.000000,28519.000000
mean,4.332423e+06,2007.010627,5024.659816,0.754786,0.392721
std,2.161092e+06,4.326800,2860.157458,0.145008,0.488364
min,2.800000e+01,2000.000000,100.000000,0.510000,0.000000
25%,2.324856e+06,2003.000000,2530.000000,0.630000,0.000000
50%,5.981915e+06,2007.000000,5010.000000,0.750000,0.000000
75%,5.990992e+06,2011.000000,7490.000000,0.880000,1.000000
max,6.000000e+06,2014.000000,10000.000000,1.000000,1.000000


In [10]:
knowledge_train_data.head()

,ID,专利,商标,著作权
0,28,0.0,1.0,1.0
1,230,0.0,0.0,0.0
2,693,0.0,0.0,0.0
3,990,0.0,0.0,0.0
4,1274,0.0,0.0,0.0


In [11]:
knowledge_train_data.describe()

,ID,专利,商标,著作权
count,2.851900e+04,28233.000000,28216.00000,28237.000000
mean,4.332423e+06,0.342507,0.36334,0.371428
std,2.161092e+06,0.474557,0.48097,0.483195
min,2.800000e+01,0.000000,0.00000,0.000000
25%,2.324856e+06,0.000000,0.00000,0.000000
50%,5.981915e+06,0.000000,0.00000,0.000000
75%,5.990992e+06,1.000000,1.00000,1.000000
max,6.000000e+06,1.000000,1.00000,1.000000


#### combine base and knowledge training data

In [12]:
combined_base_knowledge_train_data = combined_base_knowledge_data(base_train_data, knowledge_train_data)
combined_base_knowledge_train_data.head()

,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag,专利,商标,著作权
ID,,,,,,,,,,,
5986361,2014.0,7090.0,服务业,湖北,有限责任公司,自然人,0.93,0,0.0,0.0,0.0
5991749,2007.0,5940.0,零售业,湖南,合伙企业,企业法人,0.57,0,1.0,1.0,0.0
5998154,2002.0,9720.0,工业,福建,合伙企业,自然人,0.74,0,1.0,1.0,0.0
5984390,2000.0,4800.0,商业服务业,山东,股份有限公司,NaN,0.90,0,0.0,0.0,0.0
5980535,2004.0,4530.0,零售业,广东,农民专业合作社,自然人,0.95,0,0.0,1.0,1.0


### year and money data

In [13]:
year_train_data.head(10)

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,纳税总额,所有者权益合计
0,28,2015.0,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,0.0,-12300.0
1,230,2015.0,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,0.0,13440.0
2,693,2015.0,534.0,133760.0,125400.0,655424.0,262169.6,196627.2,0.0,8360.0
3,990,2015.0,863.0,33760.0,25320.0,145168.0,58067.2,14516.8,0.0,8440.0
4,1274,2015.0,254.0,74900.0,104325.0,277130.0,110852.0,55426.0,0.0,-29425.0
5,1560,2015.0,491.0,105000.0,98000.0,147000.0,73500.0,29400.0,0.0,7000.0
6,3261,2015.0,799.0,417000.0,822880.0,1751400.0,1401120.0,350280.0,0.0,-405880.0
7,3313,2015.0,784.0,501600.0,986480.0,2156880.0,1294128.0,431376.0,0.0,-484880.0
8,3537,2015.0,647.0,17800.0,13350.0,8900.0,4450.0,2670.0,0.0,4450.0
9,3719,2015.0,369.0,317000.0,465990.0,380400.0,228240.0,152160.0,0.0,-148990.0


In [14]:
year_train_data.describe()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,纳税总额,所有者权益合计
count,8.554800e+04,84692.000000,84743.000000,84621.000000,8.472000e+04,8.467200e+04,8.470300e+04,8.469900e+04,8.473100e+04,84673.000000
mean,4.332626e+06,2015.999870,510.808421,135008.595502,1.623358e+05,3.441710e+05,2.062644e+05,1.027939e+05,7.079659e+04,-27390.880033
std,2.160933e+06,0.816398,283.129690,135222.263523,1.958622e+05,4.413585e+05,2.769529e+05,1.536672e+05,1.588261e+05,108355.730296
min,2.800000e+01,2015.000000,20.000000,100.000000,0.000000e+00,6.400000e+01,2.560000e+01,7.800000e+00,0.000000e+00,-828340.000000
25%,2.325192e+06,2015.000000,266.000000,36300.000000,3.304000e+04,5.850000e+04,3.330000e+04,1.396755e+04,0.000000e+00,-53130.000000
50%,5.981916e+06,2016.000000,512.000000,88800.000000,9.126000e+04,1.764000e+05,1.027520e+05,4.514400e+04,1.240200e+03,250.000000
75%,5.990992e+06,2017.000000,756.000000,190080.000000,2.137225e+05,4.495545e+05,2.643214e+05,1.238400e+05,6.668040e+04,8900.000000
max,5.999999e+06,2017.000000,1000.000000,849150.000000,1.676640e+06,4.060875e+06,2.958000e+06,1.807398e+06,2.089620e+06,429570.000000


In [15]:
money_train_data.head()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,28,2015.0,0.0,0.0,0.0,0.000,21648.0,1298.88,0.0,0.000
1,230,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,470.4,28.224
2,693,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,5350.4,321.024
3,990,NaN,0.0,0.0,0.0,0.000,0.0,0.00,675.2,40.512
4,1274,2015.0,0.0,0.0,11085.2,443.408,0.0,0.00,NaN,0.000


In [16]:
money_train_data.describe()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
count,8.554800e+04,84703.000000,84739.000000,84667.000000,84699.00000,84672.000000,8.474000e+04,84720.000000,84686.000000,84695.000000
mean,4.332626e+06,2016.000378,3353.349261,268.717215,5115.09531,204.777317,2.596272e+04,1555.894230,1020.851124,61.231978
std,2.160933e+06,0.816496,8883.814614,711.049430,17870.10384,715.756218,8.025768e+04,4811.138407,3000.062130,179.871750
min,2.800000e+01,2015.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,2.325192e+06,2015.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,5.981916e+06,2016.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,5.990992e+06,2017.000000,0.000000,0.000000,0.00000,0.000000,2.673000e+02,10.719000,41.000000,2.520000
max,5.999999e+06,2017.000000,84830.000000,6786.400000,303922.50000,12156.900000,1.215432e+06,72925.920000,39720.000000,2383.200000


#### year data fill na

In [17]:
# fill null
new_year_train_data = year_fill_na(year_train_data)
new_year_train_data.set_index(['ID', 'year'])

new_year_train_data.head()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,纳税总额,所有者权益合计
0,28,2015.0,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,0.0,-12300.0
1,230,2015.0,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,0.0,13440.0
2,693,2015.0,534.0,133760.0,125400.0,655424.0,262169.6,196627.2,0.0,8360.0
3,990,2015.0,863.0,33760.0,25320.0,145168.0,58067.2,14516.8,0.0,8440.0
4,1274,2015.0,254.0,74900.0,104325.0,277130.0,110852.0,55426.0,0.0,-29425.0


#### money data fill na

In [18]:
# fill null
new_money_train_data = year_fill_na(money_train_data)
new_money_train_data.set_index(['ID', 'year'])

# new_money_train_data.head(8)
new_money_train_data.head()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,28,2015.0,0.0,0.0,0.0,0.000,21648.0,1298.88,0.0,0.000
1,230,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,470.4,28.224
2,693,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,5350.4,321.024
3,990,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,675.2,40.512
4,1274,2015.0,0.0,0.0,11085.2,443.408,0.0,0.00,0.0,0.000


#### Merge new year and money data

In [19]:
# Merge new year and money data
combined_new_year_money_train_data = combined_new_year_money_data(new_year_train_data, new_money_train_data)
combined_new_year_money_train_data.head()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,纳税总额,所有者权益合计,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,28,2015.0,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,0.0,-12300.0,0.0,0.0,0.0,0.000,21648.0,1298.88,0.0,0.000
1,230,2015.0,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,0.0,13440.0,0.0,0.0,0.0,0.000,0.0,0.00,470.4,28.224
2,693,2015.0,534.0,133760.0,125400.0,655424.0,262169.6,196627.2,0.0,8360.0,0.0,0.0,0.0,0.000,0.0,0.00,5350.4,321.024
3,990,2015.0,863.0,33760.0,25320.0,145168.0,58067.2,14516.8,0.0,8440.0,0.0,0.0,0.0,0.000,0.0,0.00,675.2,40.512
4,1274,2015.0,254.0,74900.0,104325.0,277130.0,110852.0,55426.0,0.0,-29425.0,0.0,0.0,11085.2,443.408,0.0,0.00,0.0,0.000


### merge splited data

In [20]:
splited_year_money_train_data = split_data(combined_new_year_money_train_data)
splited_year_money_train_data.head()

,从业人数_2015,资产总额_2015,负债总额_2015,营业总收入_2015,主营业务收入_2015,利润总额_2015,纳税总额_2015,所有者权益合计_2015,债权融资额度_2015,债权融资成本_2015,...,纳税总额_2017,所有者权益合计_2017,债权融资额度_2017,债权融资成本_2017,股权融资额度_2017,股权融资成本_2017,内部融资和贸易融资额度_2017,内部融资和贸易融资成本_2017,项目融资和政策融资额度_2017,项目融资和政策融资成本_2017
ID,,,,,,,,,,,,,,,,,,,,,
28,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,0.0,-12300.0,0.0,0.0,...,0.0,-77900.0,0.0,0.0,3444.0,137.760,0.0,0.000,0.0,0.0
230,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,0.0,13440.0,0.0,0.0,...,0.0,-70560.0,0.0,0.0,3024.0,120.960,0.0,0.000,0.0,0.0
693,534.0,133760.0,125400.0,655424.0,262169.6,196627.2,0.0,8360.0,0.0,0.0,...,0.0,54340.0,0.0,0.0,32102.4,1284.096,0.0,0.000,0.0,0.0
990,863.0,33760.0,25320.0,145168.0,58067.2,14516.8,0.0,8440.0,0.0,0.0,...,0.0,-25320.0,0.0,0.0,0.0,0.000,111661.2,6699.672,0.0,0.0
1274,254.0,74900.0,104325.0,277130.0,110852.0,55426.0,0.0,-29425.0,0.0,0.0,...,0.0,-205975.0,42800.0,3424.0,0.0,0.000,0.0,0.000,0.0,0.0


## combine all data to train

In [21]:
train = pd.merge(combined_base_knowledge_train_data, 
                 splited_year_money_train_data, 
                 on=['ID'])

In [22]:
train.head()

,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag,专利,商标,...,纳税总额_2017,所有者权益合计_2017,债权融资额度_2017,债权融资成本_2017,股权融资额度_2017,股权融资成本_2017,内部融资和贸易融资额度_2017,内部融资和贸易融资成本_2017,项目融资和政策融资额度_2017,项目融资和政策融资成本_2017
ID,,,,,,,,,,,,,,,,,,,,,
5986361,2014.0,7090.0,服务业,湖北,有限责任公司,自然人,0.93,0,0.0,0.0,...,37435.2,-70900.0,8508.0,680.64,0.0,0.0,0.0,0.0,0.0,0.0
5991749,2007.0,5940.0,零售业,湖南,合伙企业,企业法人,0.57,0,1.0,1.0,...,160380.0,-166320.0,0.0,0.00,0.0,0.0,80190.0,4811.4,0.0,0.0
5998154,2002.0,9720.0,工业,福建,合伙企业,自然人,0.74,0,1.0,1.0,...,291600.0,250.0,0.0,0.00,145800.0,5832.0,0.0,0.0,0.0,0.0
5984390,2000.0,4800.0,商业服务业,山东,股份有限公司,NaN,0.90,0,0.0,0.0,...,65280.0,165600.0,0.0,0.00,0.0,0.0,48960.0,2937.6,0.0,0.0
5980535,2004.0,4530.0,零售业,广东,农民专业合作社,自然人,0.95,0,0.0,1.0,...,0.0,-81540.0,17667.0,1413.36,0.0,0.0,0.0,0.0,0.0,0.0


## Save train

In [23]:
train.to_csv("data/train/train.csv")

In [24]:
train.describe()

,注册时间,注册资本,控制人持股比例,flag,专利,商标,著作权,从业人数_2015,资产总额_2015,负债总额_2015,...,纳税总额_2017,所有者权益合计_2017,债权融资额度_2017,债权融资成本_2017,股权融资额度_2017,股权融资成本_2017,内部融资和贸易融资额度_2017,内部融资和贸易融资成本_2017,项目融资和政策融资额度_2017,项目融资和政策融资成本_2017
count,28516.000000,28516.000000,28516.000000,28516.000000,28516.000000,28516.000000,28516.000000,28516.000000,28516.00000,2.851600e+04,...,2.851600e+04,28516.000000,28516.000000,28516.000000,28516.000000,28516.000000,2.851600e+04,28516.000000,28516.000000,28516.000000
mean,2007.010836,5024.064385,0.754743,0.392692,0.339108,0.359517,0.367794,511.746563,119299.86604,1.425609e+05,...,7.794321e+04,-31054.918642,3647.518095,291.400463,5667.460849,226.713881,2.850957e+04,1712.928322,1115.153493,67.013075
std,4.304831,2844.938780,0.144249,0.488358,0.473415,0.479867,0.482213,281.742345,124479.32416,1.794712e+05,...,1.702224e+05,117106.108417,9425.145970,753.385070,19198.418658,768.471297,8.651585e+04,5202.890113,3229.079861,193.992216
min,2000.000000,100.000000,0.510000,0.000000,0.000000,0.000000,0.000000,20.000000,100.00000,0.000000e+00,...,0.000000e+00,-828340.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,2003.000000,2560.000000,0.630000,0.000000,0.000000,0.000000,0.000000,271.000000,28840.00000,2.568000e+04,...,0.000000e+00,-62211.250000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,2007.000000,5010.000000,0.750000,0.000000,0.000000,0.000000,0.000000,512.000000,77070.00000,7.709750e+04,...,1.240200e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,2011.000000,7470.000000,0.880000,1.000000,1.000000,1.000000,1.000000,754.000000,167410.00000,1.852650e+05,...,7.670400e+04,8880.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
max,2014.000000,10000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1000.000000,750000.00000,1.463720e+06,...,2.010624e+06,429570.000000,84830.000000,6786.400000,287287.500000,11491.500000,1.215432e+06,72925.920000,38930.000000,2335.800000


## Test Data Set

### Load original testing data

In [25]:
# load testing data
base_test_data = pd.read_csv("data/test/base-test.csv")
year_test_data = pd.read_csv("data/test/year-test.csv")
knowledge_test_data = pd.read_csv("data/test/knowledge-test.csv")
money_test_data = pd.read_csv("data/test/money-test.csv")

### base and knowledge data

In [26]:
# base_test_data.head()
# base_test_data.describe()
# knowledge_test_data.head()

#### combine base and knowledge training data

In [27]:
combined_base_knowledge_test_data = combined_base_knowledge_data(base_test_data, knowledge_test_data)
combined_base_knowledge_test_data.describe()

,注册时间,注册资本,控制人持股比例,专利,商标,著作权
count,7132.000000,7132.000000,7132.000000,7132.000000,7132.000000,7132.000000
mean,2007.077257,5039.914470,0.754799,0.342821,0.358385,0.373528
std,4.321929,2839.476208,0.143953,0.474686,0.479560,0.483774
min,2000.000000,100.000000,0.510000,0.000000,0.000000,0.000000
25%,2003.000000,2640.000000,0.630000,0.000000,0.000000,0.000000
50%,2007.000000,5040.000000,0.750000,0.000000,0.000000,0.000000
75%,2011.000000,7450.000000,0.880000,1.000000,1.000000,1.000000
max,2014.000000,10000.000000,1.000000,1.000000,1.000000,1.000000


### year and money data

In [28]:
year_test_data.describe()
money_test_data.describe()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
count,2.139600e+04,21168.000000,21173.000000,21185.000000,21188.000000,21183.000000,2.120000e+04,21183.000000,21170.000000,21167.000000
mean,4.332655e+06,2016.000661,3243.435885,260.012805,5005.051121,200.682928,2.702190e+04,1614.703645,1036.034955,62.366502
std,2.163020e+06,0.816400,8750.724756,701.048040,17489.497306,700.604837,8.306795e+04,4972.506102,3064.546443,184.264032
min,4.290000e+02,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,2.331607e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,5.981952e+06,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,5.990780e+06,2017.000000,0.000000,0.000000,0.000000,0.000000,7.704000e+02,41.364000,34.700000,2.133000
max,5.999998e+06,2017.000000,80410.000000,6432.800000,311400.000000,12456.000000,1.257150e+06,75429.000000,40970.000000,2458.200000


#### year data fill na

In [29]:
# fill null
new_year_test_data = year_fill_na(year_test_data)
new_year_test_data.set_index(['ID', 'year'])

new_year_test_data.describe()

print("unique ID count in base data:", base_test_data["ID"].nunique())
print("2015 unique ID count in year data:", new_year_test_data["ID"].loc[new_year_test_data.year==2015].nunique())
print("2016 unique ID count in year data:", new_year_test_data["ID"].loc[new_year_test_data.year==2016].nunique())
print("2017 unique ID count in year data:", new_year_test_data["ID"].loc[new_year_test_data.year==2017].nunique())

unique ID count in base data: 7132
2015 unique ID count in year data: 7132
2016 unique ID count in year data: 7132
2017 unique ID count in year data: 7132


#### money data fill na

In [30]:
# fill null
new_money_test_data = year_fill_na(money_test_data)
new_money_test_data.set_index(['ID', 'year'])

# new_money_train_data.head(8)
new_money_test_data.describe()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
count,2.139600e+04,21396.000000,21396.000000,21396.000000,21396.000000,21396.000000,2.139600e+04,21396.000000,21396.000000,21396.000000
mean,4.332655e+06,2016.000000,3209.631146,257.448648,4956.394801,198.685103,2.677436e+04,1598.629057,1025.091606,61.698997
std,2.163020e+06,0.816516,8711.231069,698.055589,17411.201243,697.393302,8.272665e+04,4950.288392,3050.157292,183.387577
min,4.290000e+02,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,2.331607e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,5.981952e+06,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,5.990780e+06,2017.000000,0.000000,0.000000,0.000000,0.000000,2.565000e+02,0.850500,0.000000,0.000000
max,5.999998e+06,2017.000000,80410.000000,6432.800000,311400.000000,12456.000000,1.257150e+06,75429.000000,40970.000000,2458.200000


In [31]:
print("unique ID count in base data:", base_test_data["ID"].nunique())
print("2015 unique ID count in money data:", new_money_test_data["ID"].loc[new_money_test_data.year==2015].nunique())
print("2016 unique ID count in money data:", new_money_test_data["ID"].loc[new_money_test_data.year==2016].nunique())
print("2017 unique ID count in money data:", new_money_test_data["ID"].loc[new_money_test_data.year==2017].nunique())

unique ID count in base data: 7132
2015 unique ID count in money data: 7132
2016 unique ID count in money data: 7132
2017 unique ID count in money data: 7132


#### Merge new year and money data

In [32]:
combined_new_year_money_test_data = combined_new_year_money_data(new_year_test_data, new_money_test_data)
combined_new_year_money_test_data.head()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,纳税总额,所有者权益合计,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,429,2015.0,136.0,193400.0,183730.0,502840.0,351988.0,45342.0,0.0,9670.0,19340.0,1547.2,0.00,0.0000,0.0,0.000,0.0,0.00
1,727,2015.0,375.0,366240.0,536280.0,402864.0,282004.8,161145.6,0.0,-170040.0,0.0,0.0,32229.12,1289.1648,0.0,0.000,0.0,0.00
2,1137,2015.0,289.0,87200.0,40320.0,81536.0,57075.2,32614.4,0.0,-11200.0,0.0,0.0,0.00,0.0000,24460.8,1467.648,0.0,0.00
3,1873,2015.0,889.0,229320.0,222950.0,137592.0,55036.8,55036.8,0.0,6370.0,0.0,0.0,11007.36,440.2944,0.0,0.000,0.0,0.00
4,2260,2015.0,689.0,225750.0,325080.0,1128750.0,564375.0,338625.0,0.0,-99330.0,0.0,0.0,0.00,0.0000,0.0,0.000,11287.5,677.25


In [33]:
print("unique ID count in combined data:", combined_new_year_money_test_data["ID"].nunique())

unique ID count in combined data: 7132


### merge splited data

In [34]:
splited_year_money_test_data = split_data(combined_new_year_money_test_data)
splited_year_money_test_data.head()

,从业人数_2015,资产总额_2015,负债总额_2015,营业总收入_2015,主营业务收入_2015,利润总额_2015,纳税总额_2015,所有者权益合计_2015,债权融资额度_2015,债权融资成本_2015,...,纳税总额_2017,所有者权益合计_2017,债权融资额度_2017,债权融资成本_2017,股权融资额度_2017,股权融资成本_2017,内部融资和贸易融资额度_2017,内部融资和贸易融资成本_2017,项目融资和政策融资额度_2017,项目融资和政策融资成本_2017
ID,,,,,,,,,,,,,,,,,,,,,
429,136.0,193400.0,183730.0,502840.0,351988.0,45342.0,0.0,9670.0,19340.0,1547.2,...,0.0,9670.0,0.0,0.00,79371.36,3174.8544,0.0,0.000,0.0,0.00
727,375.0,366240.0,536280.0,402864.0,282004.8,161145.6,0.0,-170040.0,0.0,0.0,...,0.0,-335720.0,0.0,0.00,100454.40,4018.1760,0.0,0.000,0.0,0.00
1137,289.0,87200.0,40320.0,81536.0,57075.2,32614.4,0.0,-11200.0,0.0,0.0,...,0.0,-47040.0,0.0,0.00,0.00,0.0000,0.0,0.000,1008.0,60.48
1873,889.0,229320.0,222950.0,137592.0,55036.8,55036.8,0.0,6370.0,0.0,0.0,...,0.0,6370.0,0.0,0.00,0.00,0.0000,12612.6,756.756,0.0,0.00
2260,689.0,225750.0,325080.0,1128750.0,564375.0,338625.0,0.0,-99330.0,0.0,0.0,...,1116.0,-18060.0,6321.0,505.68,0.00,0.0000,0.0,0.000,0.0,0.00


### merge all data to test

In [35]:
test = pd.merge(combined_base_knowledge_test_data, 
                 splited_year_money_test_data, 
                 on=['ID'])

In [36]:
test.head()

,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,著作权,...,纳税总额_2017,所有者权益合计_2017,债权融资额度_2017,债权融资成本_2017,股权融资额度_2017,股权融资成本_2017,内部融资和贸易融资额度_2017,内部融资和贸易融资成本_2017,项目融资和政策融资额度_2017,项目融资和政策融资成本_2017
ID,,,,,,,,,,,,,,,,,,,,,
5991927,2010.0,8790.0,工业,湖南,股份有限公司,企业法人,0.64,0.0,0.0,0.0,...,0.0,-140640.0,0.0,0.0,0.0,0.000,341491.5,20489.49,0.0,0.000
5998351,2005.0,270.0,服务业,山东,股份有限公司,自然人,0.59,1.0,0.0,1.0,...,18662.4,-2835.0,0.0,0.0,0.0,0.000,0.0,0.00,194.4,11.664
5992703,2012.0,230.0,服务业,湖北,农民专业合作社,企业法人,0.52,1.0,1.0,1.0,...,414.0,-1840.0,0.0,0.0,41.4,1.656,0.0,0.00,0.0,0.000
5979231,2003.0,5980.0,商业服务业,广东,合伙企业,企业法人,0.92,0.0,0.0,0.0,...,1116.0,-116610.0,0.0,0.0,0.0,0.000,75348.0,4520.88,0.0,0.000
5995422,2007.0,160.0,工业,山东,有限责任公司,自然人,0.70,0.0,1.0,1.0,...,5904.0,160.0,0.0,0.0,0.0,0.000,8856.0,531.36,0.0,0.000


In [37]:
test.to_csv("data/test/test.csv")